# Preprocessing

In [ ]:
import cv2,os

data_path='dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

img_size=100
data=[]
target=[]
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

# Training our model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [ ]:
history=model.fit(train_data,train_target,epochs=5,validation_split=0.2)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_data,test_target)

# Testing our model

In [ ]:
import cv2

In [ ]:
faceCascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
labels=['GOOD','BAD']
colors=[(0,255,0),(255,0,0)]

In [ ]:
while True:
    ret, frame= cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=faceCascade.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        face=gray[y:y+h,x:x+w]
        img=cv2.resize(face,(100,100))
        cv2.imshow('face',face)
        cv2.imshow('face 2',img)
        img_norm=img/255.0
        img_reshap=np.reshape(img_norm,(1,100,100,1))
        result=model.predict(img_reshap)
        label=np.argmax(result)
        cv2.rectangle(frame,(x,y),(x+w,y+h),colors[label],4)
        cv2.putText(frame,labels[label],(x,y-15),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    cv2.imshow('Video',frame)
    if(cv2.waitKey(1)==27):
        break
cv2.destroyAllWindows()
cap.release()